# Team Ratings

In [4]:
# Local libraries
import Tools.ratings_utils as ru

FILENAME = "Data/data_2024.json"
TOURNAMENT_FILENAME = "Data/tournament_2024.csv"


score_df = ru.set_rating_data_frame(filename=FILENAME)

## Massey Ratings

In [ ]:
massey_ratings = ru.calculate_massey_ratings(score_df=score_df,
                                             debug=False)

ru.simulate_tournament(filename=TOURNAMENT_FILENAME,
                       ratings=massey_ratings)

# Colley Ratings

In [ ]:
colley_ratings = ru.calculate_colley_ratings(score_df=score_df,
                                             debug=False)

ru.simulate_tournament(filename=TOURNAMENT_FILENAME,
                       ratings=colley_ratings)


# Elo Ratings

In [13]:
import pandas as pd
import numpy as np


def expected_outcome(r1, r2):
    """Calculate expected probability of team 1 winning against team 2."""
    return 1 / (1 + 10 ** ((r2 - r1) / 400))


def update_elo(r1, r2, outcome, K=30):
    """Update Elo ratings after a game."""
    E1 = expected_outcome(r1, r2)
    E2 = expected_outcome(r2, r1)

    r1_new = r1 + K * (outcome - E1)
    r2_new = r2 + K * ((1 - outcome) - E2)

    return r1_new, r2_new


def compute_elo_ratings(score_df: pd.DataFrame, initial_rating=1500, K=30, debug: bool=False):
    """
    Computes Elo ratings for teams based on game results.

    Args:
        score_df (pd.DataFrame): Massey score data frame
        initial_rating: Starting rating for all teams (default 1500)
        K: Rating adjustment factor (default 30)
        debug (bool): flag to print debug statements

    Returns:
        elo_ratings: Dictionary with team Elo ratings
    """
    
    # Get unique teams and index them
    teams = list(set(score_df["Home"]).union(set(score_df["Away"])))
    elo_ratings = {team: initial_rating for team in teams}

    for _, row in score_df.iterrows():
        t1, t2, winner = row["Home"], row["Away"], row["Winner"]
        
        # Assign outcome (1 if t1 wins, 0 if t2 wins)
        outcome = 1 if winner == t1 else 0

        # Update Elo ratings
        elo_ratings[t1], elo_ratings[t2] = update_elo(r1=elo_ratings[t1], r2=elo_ratings[t2],
                                                      outcome=outcome, K=K)

    # Sort and display rankings
    elo_rankings = sorted(elo_ratings.items(), key=lambda x: x[1], reverse=True)

    if debug:
        for rank, (team, rating) in enumerate(elo_rankings, 1):
            print(f"{rank}. {team}: {rating:.2f}")
    
    return elo_ratings


elo_ratings = compute_elo_ratings(score_df=score_df,
                                  debug=False)

ru.simulate_tournament(filename=TOURNAMENT_FILENAME,
                       ratings=elo_ratings)

Round: 1 / Round of 64 - Correct picks: 12 out of 32
Round: 2 / Round of 32 - Correct picks: 0 out of 16
Round: 3 / Sweet 16 - Correct picks: 0 out of 8
Round: 4 / Elite 8 - Correct picks: 0 out of 4
Round: 5 / Final 4 - Correct picks: 0 out of 2
Round: 6 / Championship - Correct picks: 0 out of 1

Total correct picks in tournament: 12 out of 63


# Test code

In [ ]:
# Currently unused (save for Massey, Colley, Elo Ratings)
tournament = Tournament.Tournament(url=TOURNAMENT_URL,
                                   debug=True)

In [ ]:
# # Check results manually
# import csv

# # Convert dictionary to a CSV-friendly format
# with open("teams.csv", mode="w", newline="") as file:
#     writer = csv.writer(file)

#     keys = list(tourney_dict.keys())
#     writer.writerow(keys)  # Header

#     for i in range(len(tourney_dict[keys[0]])):
#         row = []
#         for key in keys:
#             row.append(tourney_dict[key][i])
#         writer.writerow(row)  # Combine team name with stats